In [ ]:
%matplotlib inline

In [ ]:
from __future__ import division, print_function
from ipywidgets import interact, FloatSlider
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('white')
import numpy as np
import pandas

import cooler
import lavaburst
import mirnylib.plotting  # likes to say hi

In [ ]:
!wget ftp://levsha.mit.edu/ftp/coolers/hg19/Dixon2015-H1hESC_ES-HindIII-allreps-filtered.5kb.cool

In [ ]:
c = cooler.Cooler('Dixon2015-H1hESC_ES-HindIII-allreps-filtered.5kb.cool')

In [ ]:
region = 'chr16:12,000,000-16,000,000'
mat = c.matrix().fetch(region)
bias = c.bintable().fetch(region)['weight'].values
mat.data = bias[mat.row] * bias[mat.col] * mat.data
A = mat.toarray()
A[np.isnan(A)] = 0
n = A.shape[0]

In [ ]:
@interact(gamma=FloatSlider(value=2, min=0, max=50, step=0.5),
          logbeta=FloatSlider(value=6, min=-6, max=8, step=1))
def _(gamma, logbeta):
    S = lavaburst.scoring.modularity_score(A, gamma=gamma, trim_diags=0)
    ranked_scores = np.array(sorted(S.ravel())[::-1])
    thresh = np.percentile(ranked_scores[ranked_scores>=0], 50)
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(ranked_scores)
    ax.axhline(0, c='k')
    ax.axhline(thresh, c='r')
    ax.set_xlabel('domain rank')
    ax.set_ylabel('domain score')

    model = lavaburst.SegModel(S)    
    # T = 0
    segs = model.optimal_segmentation()
    scores = np.array([S[a,b] for a,b in segs])
    # finite T
    beta = 10**logbeta
    prob = model.boundary_marginals(beta)

    # heatmaps
    At = lavaburst.utils.tilt_heatmap(A, n_diags=500)
    St = lavaburst.utils.tilt_heatmap(S, n_diags=500)
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(111)
    ax.imshow(np.log10(At), interpolation='none', cmap='fall', vmax=2.,
              extent=(0, n, -500, 0))
    ax.imshow(St, interpolation='none', cmap='fall', vmin=0,
              extent=(0, n, 500, 0))
    ax.set_aspect(0.5)
    ax.set_xlim([0, A.shape[0]])
    ax.set_ylim([-500, 500])

    # plot optimal domains
    for (a,b), s in zip(segs,scores):
        color = 'r' if s <= thresh else 'k'
        ax.plot([a, (a+b)/2, b], [0, (b-a), 0], color=color, lw=1)
    
    # plot boundary probs
    ax.plot(prob*200)